### Notebook to perform Zernike polynominal analysis of SKA1-LOW and LOFAR calibration

In [ ]:
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
#pylab.rcParams['image.cmap'] = 'rainbow'

from see import see

from telutil import *

import subprocess
print "Working from Git repository %s" % subprocess.check_output(["git",  "describe", "--long", "--all"])

def timestamp():
    from datetime import datetime
    print "%s %s" % (datetime.now().date(), datetime.now().time())
    

def trials(ntrials,nsources,config,name,rmin=0,rmax=200.0,tiono=10,tobs=10000*3600.0,snrthreshold=5.0):
    tp=TelPiercings()

    timestamp()

    # Minimum SNR=10 per pierce point
    for trial in range(ntrials):
        ts=TelSources()
        ts.construct(nsources=nsources, radius=wavelength/stationdiameter)


        tp.construct(ts,config,hiono=hiono,rmin=rmin,rmax=HWZ)
        if trial==0:
            tp.plot(rcore=HWZ, rmax=2.0*HWZ)

        s=numpy.sqrt(tp.assess(nnoll=nnoll, rmax=HWZ, doplot=(trial==0)))
        print "Trial %d, max(s)=%.1f" % (trial, numpy.max(s))

        if trial==0:
            Save=s/float(ntrials)
        else:
            Save=Save+s/float(ntrials)
    J=len(Save[Save>snrthreshold])
    if J<1:
        J=1
    DR=TelIono().dr(J,tobs=tobs)
    tobs=TelIono().tobs(J,DR=1e5)

    return {'dr':DR, 's':Save, 'J':J, 'tobs':tobs}

random.seed(781490893)

rhalo=40.0
nnoll=1000
wavelength=3.0
stationdiameter=35.0
rcore=0.5
rmin=0.0
rmax=40.0
hiono=300.0
HWZ=hiono*wavelength/stationdiameter # Half width to Zero
print "FOV (HWZ) at ionosphere=%.1f km" % HWZ

### Define all configs and plot

In [ ]:

stats={}
tel={}
mst={}
#configs = ['LOWBD2', 'LOWBD2-CORE', 'LOWBD2-HALO', 'LOWBD2-RASTERHALO',
#                'LOWBD2-RASTERHALO25KM', 'LOFAR']

configs = ['LOWBD2','LOWBD2-CORE', 'LOWBD2-RASTERHALO','LOWBD2-RASTERHALO25KM']

for config in configs:


# Define all config variants
    tel[config]=TelArray()
    if config=='LOWBD2':
        tel[config].readKML(kmlfile='V4Drsst512red_2.kml')
        tel[config].saveCSV('LOWBD2.csv')
    elif config =='LOWBD2-CORE':
        tel[config].readCSV('LOWBD2-CORE', l1def='LOWBD2-CORE.csv', recenter=True)
    elif config =='LOWBD2-HALO':
        tel[config].readCSV('LOWBD2-HALO', l1def='LOWBD2-HALO.csv', recenter=True)
    elif config == 'LOWBD2-RASTERHALO':
        tel['RASTERHALO']=TelArray()
        tel['RASTERHALO'].rasterBoolardy(nhalo=346, name='LOWRASTERHALO', nstations=346, scale=1.0, \
                           rhalo=40.0, rcore=0.0)
        tel[config]=TelArray().add(tel['RASTERHALO'],tel['LOWBD2-CORE'],name=config)
    elif config == 'LOWBD2-RASTERHALO25KM':
        tel['RASTERHALO25KM']=TelArray()
        tel['RASTERHALO25KM'].rasterBoolardy(nhalo=346, name='LOWRASTERHALO25KM', nstations=346, scale=40.0/25.7, \
                           rhalo=25.7, rcore=0.0)
        tel[config]=TelArray().add(tel['RASTERHALO25KM'],tel['LOWBD2-CORE'],name=config)
    elif config =='LOFAR':
        tel[config].readLOFAR()
# Plot and save the figure        
    tel[config].plot()
    plt.show()
    plt.savefig('%s.pdf'%config)

In [ ]:
### Now assess Minimum Spanning Tree. This is a proxy for infrastructure costs.

In [ ]:
def msts():
    for config in configs:
        mst[config]=tel[config].mst()
            
msts()

### Now assess all configs for varying numbers of sources

In [ ]:
ntrials=10

sources=[5, 10]

stats={}

for nsource in sources:

    for config in configs:
        print "Processing %s %d sources" % (config, nsource)
        timestamp()


        if nsource==sources[0]:
            stats[config]={}

        stats[config][nsource]=trials(ntrials, nsource, tel[config], config)
        
timestamp()





### Now plot results for all the configs

In [ ]:
plt.clf()

for nsource in sources:
    ymax=0.1
    for config in stats.keys():
        y=stats[config][nsource]['s']
        if numpy.max(y)>ymax:
            ymax=numpy.max(y)
        x=numpy.arange(nnoll)
        plt.semilogy(x, y, label=config)
    plt.legend(loc="upper right")
    plt.xlim([1,1000])
    plt.ylim([0.1,ymax])
    plt.xlabel('Singular vector index')
    plt.ylabel('Singular value')
    plt.title('Singular value spectra for different configurations: %d source' % nsource)
    plt.show()

    plt.savefig('SingularValues_sources%d.pdf' % nsource)

In [ ]:
plt.clf()

for config in stats.keys():
    i=0
    x=numpy.zeros(len(sources))
    y=numpy.zeros(len(sources))
    for nsource in sources:
        x[i]=nsource
        y[i]=stats[config][nsource]['tobs']/(365.0*24.0*3600.0)
        i=i+1
    plt.semilogy(x, y, label=config)
plt.legend(loc="upper right")
plt.xlabel('Number of sources')
plt.xlim([0,35])
plt.ylabel('Time on-sky to reach 50dB (years)')
plt.title('Time on-sky to reach 50dB vs number of sources')
plt.show()
plt.savefig('Timeonsky.pdf')
    

In [ ]:
plt.clf()

config = 'LOWBD2'

for nsource in sources:
    ymax=0.1
    y=stats[config][nsource]['s']
    if numpy.max(y)>ymax:
        ymax=numpy.max(y)
    x=numpy.arange(nnoll)
    plt.semilogy(x, y, label='Number of sources %d' %nsource)

plt.legend(loc="upper right")
plt.xlim([1,500])
plt.ylim([0.1,ymax])
plt.xlabel('Singular vector index')
plt.ylabel('Singular value')
plt.title('Singular value spectra for different numbers of sources %s' % (config))
plt.show()
plt.savefig('%s_Sources.pdf' % config)

In [ ]:
def printstats(stats):
    print "Config, Nsources, peak, J, DR, tobs, MST" 

    for config in configs:
        for nsources in sources:
            print "%s, %d, %.1f, %d, %.1f, %.1f, %.1f" \
            % (config, nsources, stats[config][nsources]['s'][0], \
               stats[config][nsources]['J'], \
            10.0*numpy.log10(stats[config][nsources]['dr']), \
               stats[config][nsources]['tobs']/(24.0*3600.0*365.0), 
               mst[config])


printstats(stats)